<a href="https://colab.research.google.com/github/Yuma2/Lyrics-Conditioned-Neural-Melody-Generation/blob/master/%E5%89%8D%E5%87%A6%E7%90%86%E3%82%A2%E3%83%97%E3%83%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

URL = input("パスを入力してください。")
df = pd.read_csv(URL)

パスを入力してください。/content/drive/MyDrive/Colab Notebooks/Nishika中古マンション/train.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.head(3)

,Unnamed: 0,ID,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,0,1000809,中古マンション等,NaN,1101,北海道,札幌市中央区,北１条西,西１８丁目,5,...,NaN,NaN,NaN,商業地域,80.0,400.0,2019年第3四半期,改装済,NaN,6.963788
1,1,1108719,中古マンション等,NaN,1206,北海道,釧路市,浪花町,釧路,10,...,NaN,NaN,NaN,第１種住居地域,60.0,200.0,2013年第1四半期,未改装,NaN,7.000000
2,2,1028203,中古マンション等,NaN,1103,北海道,札幌市東区,北４４条東,栄町(札幌),5,...,NaN,NaN,NaN,第１種住居地域,60.0,200.0,2019年第4四半期,改装済,NaN,7.301030


# まずはシンプルに処理出来るものから。例えば、時間で大半のデータは数字だが、一部（30～60分）のように数字データになっていないものを修正する。

In [ ]:
col = df.columns
for i in col:
  print(i, df[i].dtype)

Unnamed: 0 int64
ID int64
種類 object
地域 float64
市区町村コード int64
都道府県名 object
市区町村名 object
地区名 object
最寄駅：名称 object
最寄駅：距離（分） object
間取り object
面積（㎡） object
土地の形状 float64
間口 float64
延床面積（㎡） float64
建築年 object
建物の構造 object
用途 object
今後の利用目的 object
前面道路：方位 float64
前面道路：種類 float64
前面道路：幅員（ｍ） float64
都市計画 object
建ぺい率（％） float64
容積率（％） float64
取引時点 object
改装 object
取引の事情等 object
取引価格（総額）_log float64


In [ ]:
number_type=[]
object_type= []

for i in col:
  if df[i].dtype in ["int64","float64"]:
    number_type.append(i)
  else:
    object_type.append(i)

print("定量データ:",number_type)
print("定性データ:",object_type)

定量データ: ['Unnamed: 0', 'ID', '地域', '市区町村コード', '土地の形状', '間口', '延床面積（㎡）', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '建ぺい率（％）', '容積率（％）', '取引価格（総額）_log']
定性データ: ['種類', '都道府県名', '市区町村名', '地区名', '最寄駅：名称', '最寄駅：距離（分）', '間取り', '面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '取引時点', '改装', '取引の事情等']


In [ ]:
print("ユニークな文字列は、、、")
modify_type = []
for o in object_type:
    t = 0
    for i in df[o].unique():
      try:
        i = int(i)
      except:
        t +=1
        
    print(o,":",t)
    if  t/len(df[o].unique()) <1:
      modify_type.append(o)
      print(df[o].unique())

ユニークな文字列は、、、
種類 : 1
都道府県名 : 47
市区町村名 : 619
地区名 : 12186
最寄駅：名称 : 3874
最寄駅：距離（分） : 5
['5' '10' '4' '3' '6' '9' '8' '18' '30分?60分' '1' '13' '2' '23' '2H?' nan
 '29' '7' '11' '15' '12' '19' '20' '16' '25' '0' '14' '28' '24' '27'
 '1H?1H30' '21' '17' '26' '22' '1H30?2H']
間取り : 65
面積（㎡） : 1
['85' '70' '20' '60' '55' '90' '75' '95' '115' '65' '105' '50' '45' '210'
 '25' '80' '100' '260' '35' '30' '110' '15' '40' '135' '130' '125' '120'
 '170' '150' '145' '155' '140' '160' '220' '190' '10' '165' '185' '180'
 '360' '250' '710' '200' '300' '2000㎡以上' '530' '450' '490' 65 90 55 70 20
 30 75 60 25 50 35 100 45 85 80 40 95 15 110 105 10 155 145 125 115 120
 470 300 150 140 165 160 250 135 130 600 970 440 390 175 740 460 195 270
 290 420 240 190 170 360 320 210 220 1000 185 230 180 580 490 330 860 200
 260 310 500 340 '630' '1300' '290' '690' '195' '280' '175' '330' '440'
 '340' '310' '500' '240' '590' '370' '1500' '380' '230' 990 410 350 720
 280 450 890 380 480 560 630 620 1500 400 870 820 370 430 

In [ ]:
modify_type

['最寄駅：距離（分）', '面積（㎡）']

In [ ]:
for o in modify_type:
    print(o, "はユニークが",df[o].unique(), "となっております。")
    for i in df[o].unique():
      try:
        i = int(i)
      except:
        print(i,"が文字列になっています。")
        mo = input("何に修正しますか？") 
        df[o] = df[o].replace(i,mo)
    df[o] = df[o].apply(lambda x:int(x))

最寄駅：距離（分） はユニークが ['5' '10' '4' '3' '6' '9' '8' '18' '30分?60分' '1' '13' '2' '23' '2H?' nan
 '29' '7' '11' '15' '12' '19' '20' '16' '25' '0' '14' '28' '24' '27'
 '1H?1H30' '21' '17' '26' '22' '1H30?2H'] となっております。
30分?60分 が文字列になっています。
何に修正しますか？75
2H? が文字列になっています。
何に修正しますか？120
nan が文字列になっています。
何に修正しますか？0
1H?1H30 が文字列になっています。
何に修正しますか？105
1H30?2H が文字列になっています。
何に修正しますか？105
面積（㎡） はユニークが ['85' '70' '20' '60' '55' '90' '75' '95' '115' '65' '105' '50' '45' '210'
 '25' '80' '100' '260' '35' '30' '110' '15' '40' '135' '130' '125' '120'
 '170' '150' '145' '155' '140' '160' '220' '190' '10' '165' '185' '180'
 '360' '250' '710' '200' '300' '2000㎡以上' '530' '450' '490' 65 90 55 70 20
 30 75 60 25 50 35 100 45 85 80 40 95 15 110 105 10 155 145 125 115 120
 470 300 150 140 165 160 250 135 130 600 970 440 390 175 740 460 195 270
 290 420 240 190 170 360 320 210 220 1000 185 230 180 580 490 330 860 200
 260 310 500 340 '630' '1300' '290' '690' '195' '280' '175' '330' '440'
 '340' '310' '500' '240' '590' '3

In [ ]:
print("前処理の結果、")
for o in modify_type:
  print(o,"は")
  print(sorted(df[o].unique()))
  print("となりました。")

前処理の結果、
最寄駅：距離（分） は
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 75, 105, 120]
となりました。
面積（㎡） は
[10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 760, 770, 780, 790, 800, 820, 830, 850, 860, 870, 890, 900, 940, 950, 960, 970, 980, 990, 1000, 1100, 1200, 1300, 1400, 1500, 2000]
となりました。


# 次に少し面倒なファイルを

# 最後に今の時間をファイル名に加えてcsvファイルに保存する。

In [ ]:
import datetime
d =datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9), 'JST')).strftime('%Y%m%d%H%M')
URL = URL.replace('.csv', d + 'csv')
df.to_csv(URL, index=False)